In [1]:
import pandas as pd
import numpy as np

import pickle

import warnings
warnings.filterwarnings('ignore')

import multiprocessing as mp
from pyhdf import SD
import pickle

import glob

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, median_absolute_error
from sklearn.metrics import r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler

In [3]:
MUMBAI_GEO_OBJ = pickle.load(open("./mumbai_geodata.pkl", "rb"))
MUMBAI_MASK = MUMBAI_GEO_OBJ['mumbai_mask']
MUMBAI_COORDS = MUMBAI_GEO_OBJ['mumbai_coords']

DECIMAL_PLACES = 7
PROCESS_ML_DATA = True

CITY = "Mumbai"

In [4]:
india_stations_df = pd.read_pickle('../2015-2020-pm25/india_stations.pkl')
stations = india_stations_df[india_stations_df['City'] == CITY]['StationId'].values

mcd19a2_obj = pickle.load(open("./mcd19a2.pkl", "rb"))
mcd19a2_longitude, mcd19a2_latitude = mcd19a2_obj['longitude'], mcd19a2_obj['latitude']

In [5]:
def get_nearest_point_idx(latitude, longitude, user_lat, user_lon):
        
    R = 6371000
    lat1 = np.radians(user_lat)
    lat2 = np.radians(latitude)
    delta_lat = np.radians(latitude-user_lat)
    delta_lon = np.radians(longitude-user_lon)
    a = (np.sin(delta_lat/2))*(np.sin(delta_lat/2))+(np.cos(lat1))*(np.cos(lat2))*(np.sin(delta_lon/2))*(np.sin(delta_lon/2))
    c = 2*np.arctan2(np.sqrt(a),np.sqrt(1-a))
    d = R*c
    
    x, y = np.unravel_index(d.argmin(),d.shape)
    
    return x, y

In [6]:
def get_station_names(station_id):   
    df_row = india_stations_df[india_stations_df['StationId'] == station_id]
    name, city, station_lat, station_lon = df_row.values[0][0], df_row.values[0][2], df_row.values[0][3], df_row.values[0][4]

    print("="*50)
    print(f"Name: {name}")
    print(f"City: {city}")
    print(f"Coordinates: ({station_lat}, {station_lon})")
    print("="*50)
    
    return name, city, station_lat, station_lon

In [7]:
def get_nearest_3x3_grid(data, x, y):
    
    if x < 1:
        x += 1
    if x > data.shape[0]-2:
        x -= 2
    if y < 1:
        y += 1
    if y > data.shape[1]-2:
        y -= 2  
    
    three_by_three = data[x-1:x+2,y-1:y+2]
    three_by_three = three_by_three.astype(float)
    
    not_nans = np.count_nonzero(~np.isnan(three_by_three))
    
    if not_nans == 0:
        return {
            "x": x,
            "y": y,
        }
    else:
        three_by_three_average = np.nanmean(three_by_three)
        three_by_three_std = np.nanstd(three_by_three)
        three_by_three_median = np.nanmedian(three_by_three)
        
        return {
            "x": x,
            "y": y,
            "data": three_by_three,
            "average": three_by_three_average,
            "std": three_by_three_std,
            "median": three_by_three_median
        }    

In [8]:
def nearest_lat_lon(station_lat, station_lon):
    x, y = get_nearest_point_idx(mcd19a2_latitude, mcd19a2_longitude, station_lat, station_lon)
    nearest_lon, nearest_lat = np.round(mcd19a2_longitude[x,y], 8), np.round(mcd19a2_latitude[x,y], 8)

    print("="*50)
    print(f"Nearest Coordinates: ({nearest_lat}, {nearest_lon})")
    print("="*50)
    
    return x, y, nearest_lat, nearest_lon

In [9]:
def extract_aod_values_from_hdf(idx, x_coord, y_coord):

    SDS_NAME = "Optical_Depth_047"
    FILE_NAME = FILE_LIST[idx]
    hdf = SD.SD(FILE_NAME)
    sds = hdf.select(SDS_NAME)

    NAME = FILE_NAME.split('/')[-1]

    data = sds.get()

    attributes = sds.attributes()
    scale_factor = attributes['scale_factor']
    fv = attributes['_FillValue']

    data = data.astype(float)
    data[data == fv] = np.nan
    data = np.nanmean(data, axis=0)

    scaled_data = data * scale_factor
    
    try:
        fix_station_aod = get_nearest_3x3_grid(scaled_data, x_coord, y_coord)['average'].round(3)
    except Exception as e:
        return

    aod_data = np.array(list(zip(scaled_data[MUMBAI_MASK], mcd19a2_latitude[MUMBAI_MASK], mcd19a2_longitude[MUMBAI_MASK])))
    df_aod = pd.DataFrame(aod_data, columns=['aod_value', 'latitude', 'longitude'])
    df_aod = df_aod.dropna().reset_index(drop=True).round(DECIMAL_PLACES)
    
    data = np.array(list(zip([fix_station_aod for k in range(df_aod.shape[0])], 
                             ((fix_station_lat - df_aod['latitude']).values), 
                             ((fix_station_lon - df_aod['longitude']).values), 
                             df_aod['aod_value'].values)
                        ))
    
    if (data.shape[0] == 0):
        return
    
    total_ml_data.append(data)

In [10]:
def get_polynomial_reg_model(n = 4):
    
    poly_reg = PolynomialFeatures(degree = n)
    X_poly = poly_reg.fit_transform(X_train)

    regressor = LinearRegression(n_jobs=-1)
    regressor.fit(X_poly, y_train)

    y_pred = regressor.predict(poly_reg.transform(X_test))
    score = {
        "r2_score": r2_score(y_test, y_pred),
        "MAE": mean_absolute_error(y_test, y_pred), 
        "MSLE": mean_squared_log_error(y_test, y_pred),
        "MdAbsE": median_absolute_error(y_test, y_pred),
        "MAPE": mean_absolute_percentage_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "mean": np.mean(y_test)
    }
    
    return regressor, score, poly_reg

def get_linear_reg_model():
    
    regressor = LinearRegression(n_jobs=-1)
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    score = {
        "r2_score": r2_score(y_test, y_pred),
        "MAE": mean_absolute_error(y_test, y_pred), 
        "MSLE": mean_squared_log_error(y_test, y_pred),
        "MdAbsE": median_absolute_error(y_test, y_pred),
        "MAPE": mean_absolute_percentage_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "mean": np.mean(y_test)
    }
    
    return regressor, score, None

def get_decision_tree_reg_model():
    
    regressor = DecisionTreeRegressor(random_state=42)
    regressor.fit(X_train, y_train)
    
    y_pred = regressor.predict(X_test)
    score = {
        "r2_score": r2_score(y_test, y_pred),
        "MAE": mean_absolute_error(y_test, y_pred), 
        "MSLE": mean_squared_log_error(y_test, y_pred),
        "MdAbsE": median_absolute_error(y_test, y_pred),
        "MAPE": mean_absolute_percentage_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "mean": np.mean(y_test)
    }
    
    return regressor, score, None

def get_random_forest_reg_model():
    
    regressor = RandomForestRegressor(random_state=42, n_jobs=-1)
    regressor.fit(X_train, y_train)

    y_pred = regressor.predict(X_test)
    score = {
        "r2_score": r2_score(y_test, y_pred),
        "MAE": mean_absolute_error(y_test, y_pred), 
        "MSLE": mean_squared_log_error(y_test, y_pred),
        "MdAbsE": median_absolute_error(y_test, y_pred),
        "MAPE": mean_absolute_percentage_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "mean": np.mean(y_test)
    }
    
    return regressor, score, None

def get_svr_reg_model():
    
    sc_X = StandardScaler()
    sc_y = StandardScaler()
    X_train_sc = sc_X.fit_transform(X_train)
    y_train_sc = sc_y.fit_transform(y_train)
    
    regressor = SVR(kernel='rbf')
    regressor.fit(X_train_sc, y_train_sc)
    
    y_pred = sc_y.inverse_transform(regressor.predict(sc_X.transform(X_test)))
    score = {
        "r2_score": r2_score(y_test, y_pred),
        "MAE": mean_absolute_error(y_test, y_pred), 
        "MSLE": mean_squared_log_error(y_test, y_pred),
        "MdAbsE": median_absolute_error(y_test, y_pred),
        "MAPE": mean_absolute_percentage_error(y_test, y_pred),
        "RMSE": np.sqrt(mean_squared_error(y_test, y_pred)),
        "mean": np.mean(y_test)
    }
    
    return regressor, score, (sc_X, sc_y)

In [11]:
MODEL_LIST = [
    ('Polynomial', get_polynomial_reg_model),
    ('Linear', get_linear_reg_model),
    ('Decision Tree', get_decision_tree_reg_model),
    ('Random Forest', get_random_forest_reg_model),
    ('SVR', get_svr_reg_model)
]

In [12]:
import configparser

config = configparser.ConfigParser()
config.read("./config.ini")

PATH = config['path']['hdf_path_2020'] + "*"
FILE_LIST = glob.glob(PATH)

In [13]:
if PROCESS_ML_DATA:
    
    for station_id in stations:
        
        name, city, station_lat, station_lon = get_station_names(station_id)
        x_coord, y_coord, nearest_lat, nearest_lon = nearest_lat_lon(station_lat, station_lon)
        
        station_obj = {
            "name": name,
            "city": city,
            "station_lat": station_lat,
            "station_lon": station_lon,
            "x_coord": x_coord,
            "y_coord": y_coord,
            "nearest_lat": nearest_lat,
            "nearest_lon": nearest_lon
        }
        
        def get_tuple(idx):
            return idx, x_coord, y_coord

        fix_station_lat, fix_station_lon = nearest_lat, nearest_lon 
        
        manager = mp.Manager()
        total_ml_data = manager.list()

        pool = mp.Pool(mp.cpu_count())
        pool.starmap(extract_aod_values_from_hdf, [get_tuple(idx) for idx in range(len(FILE_LIST))])
        pool.close()

        total_data = np.vstack([data for data in total_ml_data])
        np.random.shuffle(total_data)

        SIZE_LIMIT = int(2.5e4)

        if total_data.shape[0] > SIZE_LIMIT:
            total_data = total_data[np.random.choice(total_data.shape[0], int(SIZE_LIMIT), replace=False), :]

        X = total_data[:,:-1]
        y = total_data[:,-1:]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        best = {}
        best_score = 9e9

        rows = []

        for name, model_fn in MODEL_LIST:
            regressor, score, scaler = model_fn()
            row = [f"{name} Regression", score['MAE'], score['RMSE'], score['MSLE'], score['MdAbsE']]
            rows.append(row)
            
            if score['RMSE'] < best_score:
                best['station_id'] = station_obj
                best['name'] = name + " Regression"
                best['regressor'] = regressor
                best['scaler'] = scaler
                best['score'] = score
                best_score = score['RMSE']

        model_dfs = pd.DataFrame(rows, columns=["model", "MAE", "RMSE", "MSLE", "MdAbsE"])
        print(model_dfs)
        
        best['other_model_performance'] = model_dfs

        pickle.dump(best, open(f"spt_models/{station_id}_spt.pkl", "wb"), protocol=4)
        
else:
    for station_id in stations:
        get_station_names(station_id)
        best = pickle.load(open(f"spt_models/{station_id}_spt.pkl", "rb"))
        nearest_lat_lon(best['station_id']['station_lat'], best['station_id']['station_lon'])
        print(best['other_model_performance'])

Name: Borivali East, Mumbai - MPCB
City: Mumbai
Coordinates: (19.2298, 72.8609)
Nearest Coordinates: (19.23269391, 72.85953478)
                      model       MAE      RMSE      MSLE    MdAbsE
0     Polynomial Regression  0.034924  0.050615  0.001532  0.022837
1         Linear Regression  0.035527  0.052330  0.001643  0.023452
2  Decision Tree Regression  0.017152  0.036421  0.000727  0.003000
3  Random Forest Regression  0.015764  0.028737  0.000456  0.005450
4            SVR Regression  0.033348  0.050713  0.001546  0.018299
Name: Chhatrapati Shivaji Intl. Airport (T2), Mumbai - MPCB
City: Mumbai
Coordinates: (19.0895595, 72.86561440000001)
Nearest Coordinates: (19.09090909, 72.8708573)
                      model       MAE      RMSE      MSLE    MdAbsE
0     Polynomial Regression  0.026391  0.038702  0.000882  0.016128
1         Linear Regression  0.027383  0.040154  0.000955  0.015967
2  Decision Tree Regression  0.015956  0.033203  0.000630  0.003000
3  Random Forest Regression

In [14]:
table_dict = {}

for station_id in stations:
        best = pickle.load(open(f"spt_models/{station_id}_spt.pkl", "rb"))
        df = best['other_model_performance'].set_index('model')
        for model in df.index.values:
            try:
                table_dict[model] += df.loc[model]
            except:
                table_dict[model] = df.loc[model]

In [15]:
result_df = pd.DataFrame(table_dict)/len(df.index.values)
result_df = result_df.transpose()

In [16]:
result_df

,MAE,RMSE,MSLE,MdAbsE
Polynomial Regression,0.054516,0.077479,0.001991,0.036427
Linear Regression,0.055867,0.079422,0.002101,0.037029
Decision Tree Regression,0.029735,0.062625,0.001232,0.005800
Random Forest Regression,0.027526,0.049647,0.000778,0.009408
SVR Regression,0.052269,0.077193,0.001980,0.030697


In [17]:
print(result_df.round(3).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &    MAE &   RMSE &   MSLE &  MdAbsE \\
\midrule
Polynomial Regression    &  0.055 &  0.077 &  0.002 &   0.036 \\
Linear Regression        &  0.056 &  0.079 &  0.002 &   0.037 \\
Decision Tree Regression &  0.030 &  0.063 &  0.001 &   0.006 \\
Random Forest Regression &  0.028 &  0.050 &  0.001 &   0.009 \\
SVR Regression           &  0.052 &  0.077 &  0.002 &   0.031 \\
\bottomrule
\end{tabular}

